# Chapter 5: Loading and Saving your Data (Python)

In this Notebook, we will explore how to load and save data in three different formats:

    * Parquet
    * CSV
    * Json

The problems included in this notebook are solved using different high-level data sources included in Spark SQL. In addition, the problems related to `CSV` and `Json` are solved also directly using the SparkContext and RDDs.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Loading-Saving-Data").master("local[*]").getOrCreate()
sc = spark.sparkContext

## Parquet Format

Loading data

In [2]:
parquet_data = spark.read.parquet("../data/person.parquet")

In [3]:
parquet_data.show()

+----+---+
|Name|Age|
+----+---+
|Raul| 29|
|Javi| 34|
+----+---+



Saving data

In [4]:
parquet_data.write.mode("overwrite").parquet("../data/person_write.parquet")

In [5]:
spark.read.parquet("../data/person_write.parquet").show()

+----+---+
|Name|Age|
+----+---+
|Raul| 29|
|Javi| 34|
+----+---+



## CSV Format

### Original Approach

Loading data

In [53]:
sep = ","
csv_data_original = sc.textFile("../data/person.csv").map(lambda x: x.split(sep))

In [54]:
csv_data_original.take(2)

[['Name', 'Age'], ['Raul', '29']]

Write data

In [86]:
!rm -rf ../data/person_write_orginal.csv
csv_data_original.map(lambda x: str(x)[1:-1].replace("'", "").replace(" ", "")).coalesce(1).saveAsTextFile("../data/person_write_orginal.csv")

In [87]:
sep = ","
csv_data_original_loaded = sc.textFile("../data/person_write_orginal.csv").map(lambda x: x.split(sep))

In [88]:
csv_data_original_loaded.take(2)

[['Name', 'Age'], ['Raul', '29']]

### Using SQL API

Loading data

In [6]:
csv_data = spark.read.option("header", "true").option("inferschema", "true").csv("../data/person.csv")

In [7]:
csv_data.show()

+----+---+
|Name|Age|
+----+---+
|Raul| 29|
|Javi| 34|
+----+---+



In [8]:
csv_data_bis = spark.read.csv("../data/person.csv", header=True, inferSchema=True)

In [9]:
csv_data_bis.show()

+----+---+
|Name|Age|
+----+---+
|Raul| 29|
|Javi| 34|
+----+---+



In [10]:
import pyspark.sql.types as T

In [11]:
schema = T.StructType([T.StructField("Name", T.StringType(), True),
                       T.StructField("Age", T.IntegerType(), True)])

In [12]:
csv_data_schema = spark.read.csv("../data/person.csv", header=True, schema=schema)

In [13]:
csv_data_schema.show()

+----+---+
|Name|Age|
+----+---+
|Raul| 29|
|Javi| 34|
+----+---+



Writing data

In [14]:
csv_data.write.mode("overwrite").csv("../data/person_write.csv", header=True)

In [15]:
spark.read.csv("../data/person_write.csv", header=True, inferSchema=True).show()

+----+---+
|Name|Age|
+----+---+
|Raul| 29|
|Javi| 34|
+----+---+



## JSON Format

### Original Approach

Loading data

In [36]:
import json

In [45]:
json_data_original = sc.textFile("../data/person.json").map(lambda x: json.loads(x))

In [46]:
json_data_original.take(2)

[{'name': 'Raul', 'age': 29}, {'name': 'Javi', 'age': 33}]

Saving data

In [111]:
!rm -rf ../data/person_write_orginal.json
json_data_original.map(lambda x: str(x).replace("'", '"')).coalesce(1).saveAsTextFile("../data/person_write_orginal.json")

In [112]:
sep = ","
json_data_original_loaded = sc.textFile("../data/person_write_orginal.json").map(lambda x: json.loads(x))

In [113]:
json_data_original_loaded.take(2)

[{'name': 'Raul', 'age': 29}, {'name': 'Javi', 'age': 33}]

### Using SQL API

Loading data

In [16]:
json_data = spark.read.json("../data/person.json")

In [17]:
json_data.show()

+---+----+
|age|name|
+---+----+
| 29|Raul|
| 33|Javi|
+---+----+



Saving data

In [18]:
json_data.write.mode("overwrite").json("../data/person_write.json")

In [19]:
spark.read.json("../data/person_write.json").show()

+---+----+
|age|name|
+---+----+
| 29|Raul|
| 33|Javi|
+---+----+

